In [54]:
''' This file has our baseline model implementation'''
import pandas as pd
import re
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

In [55]:
# import data
digits = load_digits()


In [56]:
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.25, random_state=0)

## Logistic Regression

In [57]:
clf = LogisticRegression()

In [58]:
clf.fit(x_train, y_train)

/opt/anaconda3/envs/cs229/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [59]:
predictions = clf.predict(x_test)

In [60]:
# Accuracy Score
score = clf.score(x_test, y_test)
print(f"Logistic Regression Accuracy Score: {score}")

Logistic Regression Accuracy Score: 0.9511111111111111


In [61]:
metric_dict = classification_report(y_test, predictions, output_dict=True)

In [62]:
metric_dict

{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 37.0},
 '1': {'precision': 0.8888888888888888,
  'recall': 0.9302325581395349,
  'f1-score': 0.9090909090909091,
  'support': 43.0},
 '2': {'precision': 0.975609756097561,
  'recall': 0.9090909090909091,
  'f1-score': 0.9411764705882353,
  'support': 44.0},
 '3': {'precision': 0.9148936170212766,
  'recall': 0.9555555555555556,
  'f1-score': 0.9347826086956522,
  'support': 45.0},
 '4': {'precision': 0.9736842105263158,
  'recall': 0.9736842105263158,
  'f1-score': 0.9736842105263158,
  'support': 38.0},
 '5': {'precision': 0.9787234042553191,
  'recall': 0.9583333333333334,
  'f1-score': 0.968421052631579,
  'support': 48.0},
 '6': {'precision': 1.0,
  'recall': 0.9807692307692307,
  'f1-score': 0.9902912621359223,
  'support': 52.0},
 '7': {'precision': 0.9787234042553191,
  'recall': 0.9583333333333334,
  'f1-score': 0.968421052631579,
  'support': 48.0},
 '8': {'precision': 0.9148936170212766,
  'recall': 0.8958333

## SVM

In [63]:
from sklearn import svm

In [64]:
svc = svm.SVC(gamma=0.001, C=100)

In [65]:
svc.fit(x_train, y_train)

SVC(C=100, gamma=0.001)

In [66]:
y_pred = svc.predict(x_test)

In [67]:
# Accuracy Score
score = svc.score(x_test, y_test)
print(f"SVC Accuracy Score: {score}")

SVC Accuracy Score: 0.9933333333333333


In [68]:
metric_dict = classification_report(y_test, y_pred, output_dict=True)

In [69]:
metric_dict

{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 37.0},
 '1': {'precision': 0.9772727272727273,
  'recall': 1.0,
  'f1-score': 0.9885057471264368,
  'support': 43.0},
 '2': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 44.0},
 '3': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 45.0},
 '4': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 38.0},
 '5': {'precision': 0.9791666666666666,
  'recall': 0.9791666666666666,
  'f1-score': 0.9791666666666666,
  'support': 48.0},
 '6': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 52.0},
 '7': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 48.0},
 '8': {'precision': 1.0,
  'recall': 0.9791666666666666,
  'f1-score': 0.9894736842105263,
  'support': 48.0},
 '9': {'precision': 0.9787234042553191,
  'recall': 0.9787234042553191,
  'f1-score': 0.9787234042553191,
  'support': 47.0},
 'accuracy': 0.9933333333333333,
 'macro avg': {'precision': 0.9935162798194

## Decision Trees

In [70]:
from sklearn.tree import DecisionTreeClassifier

In [71]:
dt = DecisionTreeClassifier(criterion = 'gini')

In [73]:
dt.fit(x_train, y_train)

DecisionTreeClassifier()

In [74]:
y_pred = dt.predict(x_test)

In [76]:
score = accuracy_score(y_test, y_pred)
print(f"Decision Tree Accuracy Score: {score}")

Decision Tree Accuracy Score: 0.8377777777777777


In [77]:
metric_dict = classification_report(y_test, y_pred, output_dict=True)
metric_dict

{'0': {'precision': 0.8918918918918919,
  'recall': 0.8918918918918919,
  'f1-score': 0.8918918918918919,
  'support': 37.0},
 '1': {'precision': 0.813953488372093,
  'recall': 0.813953488372093,
  'f1-score': 0.813953488372093,
  'support': 43.0},
 '2': {'precision': 0.7391304347826086,
  'recall': 0.7727272727272727,
  'f1-score': 0.7555555555555555,
  'support': 44.0},
 '3': {'precision': 0.7,
  'recall': 0.7777777777777778,
  'f1-score': 0.7368421052631579,
  'support': 45.0},
 '4': {'precision': 0.8461538461538461,
  'recall': 0.868421052631579,
  'f1-score': 0.8571428571428571,
  'support': 38.0},
 '5': {'precision': 0.84,
  'recall': 0.875,
  'f1-score': 0.8571428571428571,
  'support': 48.0},
 '6': {'precision': 0.98,
  'recall': 0.9423076923076923,
  'f1-score': 0.9607843137254902,
  'support': 52.0},
 '7': {'precision': 0.9302325581395349,
  'recall': 0.8333333333333334,
  'f1-score': 0.8791208791208791,
  'support': 48.0},
 '8': {'precision': 0.8666666666666667,
  'recall': 

## Random Forest Classifier

In [78]:
from sklearn.ensemble import RandomForestClassifier

In [79]:
rc = RandomForestClassifier(n_estimators = 150)

In [80]:
rc.fit(x_train, y_train)

RandomForestClassifier(n_estimators=150)

In [82]:
y_pred = rc.predict(x_test)

In [84]:
score = accuracy_score(y_test, y_pred)
print(f'Random Forest Accuracy: {score}')

Random Forest Accuracy: 0.9777777777777777


In [85]:
metric_dict = classification_report(y_test, y_pred, output_dict=True)
metric_dict 

{'0': {'precision': 0.9736842105263158,
  'recall': 1.0,
  'f1-score': 0.9866666666666667,
  'support': 37.0},
 '1': {'precision': 0.9772727272727273,
  'recall': 1.0,
  'f1-score': 0.9885057471264368,
  'support': 43.0},
 '2': {'precision': 1.0,
  'recall': 0.9545454545454546,
  'f1-score': 0.9767441860465116,
  'support': 44.0},
 '3': {'precision': 0.9361702127659575,
  'recall': 0.9777777777777777,
  'f1-score': 0.9565217391304348,
  'support': 45.0},
 '4': {'precision': 1.0,
  'recall': 0.9736842105263158,
  'f1-score': 0.9866666666666667,
  'support': 38.0},
 '5': {'precision': 0.9791666666666666,
  'recall': 0.9791666666666666,
  'f1-score': 0.9791666666666666,
  'support': 48.0},
 '6': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 52.0},
 '7': {'precision': 0.96,
  'recall': 1.0,
  'f1-score': 0.9795918367346939,
  'support': 48.0},
 '8': {'precision': 0.9782608695652174,
  'recall': 0.9375,
  'f1-score': 0.9574468085106383,
  'support': 48.0},
 '9': {'precision'